In [1]:
import sys
import re
from tableausdk import *
from tableausdk.Extract import *
import os
import pandas as pd
import timeit
from rpy2 import robjects
import pandas.rpy.common as com
from rpy2.robjects import pandas2ri
pandas2ri.activate()

C:\Users\puneet.jindal\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html


In [2]:
robjects.r("load('.RData')")
print robjects.r('dim(mydata)')
df = com.load_data('mydata')
df.head()

[74182    21]


,Hotel.Name,Hotel.Code,Export.Date,Confirmation,Book.Time,CheckIn,CheckOut,Persons,Nights,Rooms,...,Rate.Code,Rate.Code.Name,Market.Segment,Channel,Sub.Channel,TA.IATA,Room.Amount,Currency,Res.Status,AdCode
1,Red Lion Hotel Anaheim Resort,CAANAH,6/6/2016,45S7DU3ZG,6/30/2014,7/3/2014,7/4/2014,2,1,1,...,BK6,Booking.com Promo 1,IN,Booking.com,,,81.75,USD,Modified,
2,Red Lion Hotel Bellevue,WABELL,6/6/2016,4HZ7DU3G0,6/30/2014,8/29/2014,8/31/2014,2,2,1,...,BK1,Booking.com BAR1,IN,Booking.com,,,225.75,USD,Modified,
3,Red Lion Hotel Anaheim Resort,CAANAH,6/6/2016,45S7DU3GE,6/30/2014,8/6/2014,8/10/2014,4,4,1,...,EX6,Expedia Promo 1,IN,Expedia,EX,,325.04,USD,Modified,
4,Red Lion Hotel Redding,CAREDD,6/6/2016,4U87DU3G2,6/30/2014,6/30/2014,7/1/2014,2,1,1,...,DAILY,Everyday Low Rate,BR,PMS,,,119.00,USD,Modified,
5,Red Lion Hotel Redding,CAREDD,6/6/2016,4U87DU3G8,6/30/2014,6/30/2014,7/1/2014,1,1,1,...,DAILY,Everyday Low Rate,BR,PMS,,,109.00,USD,Modified,


In [3]:
#new_extract.close()
#os.system('del TableauSDKExtract.log') 
#ExtractAPI.cleanup()
import re
df.iloc[74173,12]
re.sub(r'[^\x00-\x7F]+',' ', df.iloc[74173,12])

'DO NOT USE - Interface Default   DO NOT USE'

In [4]:
# create a list of column names
colnames = df.columns
# create a list of column types
coltypes = df.dtypes

fieldMap = { 
    'float64' :     Type.DOUBLE,
    'float32' :     Type.DOUBLE,
    'int64' :       Type.INTEGER,
    'int32' :       Type.INTEGER,
    'object':       Type.UNICODE_STRING,
    'bool' :        Type.BOOLEAN
}
ExtractAPI.initialize()
fname = 'mydata1.tde'
try:  
    new_extract = Extract(fname)
except: 
    os.system('del ' + fname)
    os.system('del TableauSDKExtract.log') 
    new_extract = Extract(fname)
# define the table definition
tableDef = TableDefinition()
# for each column, add the appropriate info the Table Definition
for i in range(0, len(colnames)):
    cname = colnames[i]
    ctype = fieldMap.get(str(coltypes[i]))
    tableDef.addColumn(cname, ctype)      
new_table = new_extract.addTable('Extract', tableDef)

for r in range(0, df.shape[0]):
    # Create new row
    row = Row(tableDef) 
    #print "\n"
    for c in range(0, len(coltypes)):
        if str(coltypes[c]) == 'float64':
            row.setDouble(c, df.iloc[r,c])
        elif str(coltypes[c]) == 'float32':
            row.setDouble(c, df.iloc[r,c])
        elif str(coltypes[c]) == 'int64':
            row.setLongInteger(c, df.iloc[r,c])   
        elif str(coltypes[c]) == 'int32':
            row.setInteger(c, df.iloc[r,c])
        elif str(coltypes[c]) == 'object':
            row.setString(c, re.sub(r'[^\x00-\x7F]+'," ", df.iloc[r,c]))
        elif str(coltypes[c]) == 'bool':
            row.setBoolean(c, df.iloc[r,c])
        else:
            row.setNull(c)
    new_table.insert(row)

# Close the extract in order to save the .tde file and clean up resources
new_extract.close()

ExtractAPI.cleanup()

In [18]:
ExtractAPI.initialize()

new_extract = Extract('mydata.tde')

# Create a new table definition with 3 columns
table_definition = TableDefinition()
table_definition.addColumn('id',         Type.INTEGER)        # column 0
table_definition.addColumn('name',       Type.UNICODE_STRING) # column 1
table_definition.addColumn('start_date', Type.DATE)           # column 2

new_table = new_extract.addTable('Extract', table_definition)

# Create new row
new_row = Row(table_definition)   # Pass the table definition to the constructor

# Set column values. The first parameter is the column number (its
# ordinal position) The second parameter (or second and subsequent paramaters) is 
# the value to set the column to.
new_row.setInteger(0, 1)
new_row.setString(1, 'Amanda')
new_row.setDate(2, 2014, 6, 9)
new_table.insert(new_row) # Add the new row to the table

new_row.setInteger(0, 2)
new_row.setString(1, 'Brian')
new_row.setDate(2, 2014, 10, 13)
new_table.insert(new_row)

new_row.setInteger(0, 3)
new_row.setString(1, 'Christina')
new_row.setDate(2, 2015, 2, 16)
new_table.insert(new_row)

# Close the extract in order to save the .tde file and clean up resources
new_extract.close()

ExtractAPI.cleanup()